In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn



In [53]:
# train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp32.csv' ,delimiter=',')
# test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp32.csv' ,delimiter=',')

train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')





In [54]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [55]:
# Separate features and target
X = train.drop(columns=['price'])
y = ((train['price'])) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [57]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# # Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [58]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [45]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Ensure the inputs are float tensors (required for MSELoss and logarithmic operations)

    
    # Compute the RMSLE using the logarithmic differences
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))
    # return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

In [46]:
from torch.optim.lr_scheduler import CyclicLR

def train_model(model, train_loader, test_loader, num_epochs=5):
    model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
    scheduler = CyclicLR(optimizer, base_lr=1e-4, max_lr=0.1, step_size_up=5*len(train_loader),
                     mode='triangular', cycle_momentum=True)


    min_train_rmsle = float('inf')
    min_test_rmsle = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()  # Update learning rate at each batch

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)

        if avg_train_rmsle < min_train_rmsle:
            min_train_rmsle = avg_train_rmsle
        if avg_test_rmsle < min_test_rmsle:
            min_test_rmsle = avg_test_rmsle

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , Current LR = {scheduler.get_last_lr()}')

    print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
    print(f'Minimum Test RMSLE so far: {min_test_rmsle}')


In [ ]:
# def train_model(model, train_loader, test_loader, num_epochs=5):
#     model.to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-6)

#     # Initialize minimum loss to a large value
#     min_train_rmsle = float('inf')
#     min_test_rmsle = float('inf')

#     for epoch in range(num_epochs):
#         model.train()
#         total_train_loss = 0
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.to(device), targets.to(device)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = rmsle_loss(outputs, targets)
#             loss.backward()
#             optimizer.step()

#             total_train_loss += loss.item()

#         avg_train_rmsle = total_train_loss / len(train_loader)

#         model.eval()
#         total_test_loss = 0
#         with torch.no_grad():
#             for inputs, targets in test_loader:
#                 inputs, targets = inputs.to(device), targets.to(device)
#                 outputs = model(inputs)
#                 test_loss = rmsle_loss(outputs, targets)
#                 total_test_loss += test_loss.item()

#         avg_test_rmsle = total_test_loss / len(test_loader)
        
#         # Update the learning rate scheduler
#         scheduler.step(avg_test_rmsle)

#         if avg_train_rmsle < min_train_rmsle:
#             min_train_rmsle = avg_train_rmsle
#         if avg_test_rmsle < min_test_rmsle:
#             min_test_rmsle = avg_test_rmsle

#         print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle} , {scheduler.get_last_lr()}')
    
#     print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
#     print(f'Minimum Test RMSLE so far: {min_test_rmsle}')

In [59]:
num_features = X_train.shape[1]


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedMLP(nn.Module):
    def __init__(self, num_features):
        super(AdvancedMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.1)
        
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.01)
        
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.final_layer = nn.Linear(128, 1)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = F.leaky_relu(self.bn4(self.fc4(x)))
        x = self.final_layer(x)
        return x


In [24]:
model = AdvancedMLP(num_features).to(device)
train_model(model, train_loader, test_loader, num_epochs=10)
# Minimum Train RMSLE so far: 0.6773702232499507
# Minimum Test RMSLE so far: 0.678769255297098

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester 6\461p\ECE-461P-Term-Project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1: Train RMSLE = 0.6977716390340981, Test RMSLE = 0.6808885407548207 , [0.001]
Epoch 2: Train RMSLE = 0.6822316377854593, Test RMSLE = 0.6794625739831437 , [0.001]
Epoch 3: Train RMSLE = 0.6810945735016469, Test RMSLE = 0.6795257989762026 , [0.001]
Epoch 4: Train RMSLE = 0.6804180366821999, Test RMSLE = 0.6796900673436181 , [0.001]
Epoch 5: Train RMSLE = 0.6798826915744339, Test RMSLE = 0.6788149245278002 , [0.001]
Epoch 6: Train RMSLE = 0.6795481715714801, Test RMSLE = 0.6794189264580746 , [0.001]
Epoch 7: Train RMSLE = 0.6791912301538907, Test RMSLE = 0.6789408963060204 , [0.001]
Epoch 8: Train RMSLE = 0.6788582577895129, Test RMSLE = 0.6789104209054857 , [0.0005]
Epoch 9: Train RMSLE = 0.6777819392603369, Test RMSLE = 0.678769255297098 , [0.0005]
Epoch 10: Train RMSLE = 0.6773702232499507, Test RMSLE = 0.6789821903807534 , [0.0005]
Minimum Train RMSLE so far: 0.6773702232499507
Minimum Test RMSLE so far: 0.678769255297098


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomMLP(nn.Module):
    def __init__(self, num_features):
        super(CustomMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.2)
        
        self.final_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        
        x = self.dropout(x)
        x = self.final_layer(x)
        return x


In [ ]:
model10 = CustomMLP(num_features).to(device)
train_model(model10, train_loader, test_loader, num_epochs=10)

# Minimum Train RMSLE so far: 0.6754996292528404
# Minimum Test RMSLE so far: 0.6798468978210392

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicMLP(nn.Module):
    def __init__(self, num_features, max_first_layer_size, min_layer_size=64, output_size=1):
        super(DynamicMLP, self).__init__()
        layers = []
        current_size = max_first_layer_size
        print(current_size)
        # Ensure the loop creates layers correctly by adjusting input and output sizes
        input_size = num_features  # Starting number of features
        while current_size > min_layer_size:
            next_size = max(current_size // 2, min_layer_size)
            layers.append(nn.Linear(input_size, current_size))
            layers.append(nn.BatchNorm1d(current_size))
            layers.append(nn.Dropout(0.1))  # Using a constant dropout rate
            layers.append(nn.LeakyReLU())
            input_size = current_size  # Update input size for the next layer
            current_size = next_size
        
        # Append the final layer
        layers.append(nn.Linear(input_size, output_size))

        # Register all layers
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Example of using the DynamicMLP class


In [60]:
max_first_layer_size = 8192  # Adjusted for demonstration
model = DynamicMLP(num_features, max_first_layer_size).to(device)
print(model)


8192
DynamicMLP(
  (layers): Sequential(
    (0): Linear(in_features=48, out_features=8192, bias=True)
    (1): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=8192, out_features=4096, bias=True)
    (5): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.1, inplace=False)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=4096, out_features=2048, bias=True)
    (9): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Dropout(p=0.1, inplace=False)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=2048, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout(p=0.1, inplace=False)
    (15): LeakyReLU(negative_slope=0.01)
    (16): Linear(in

In [61]:
train_model(model, train_loader, test_loader, num_epochs=10)
